In [24]:
import pprint as pp
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import missingno as msno

engineer_mode = True # True = Don't show plots etc.

mode = "train" # Use train dataset

if mode == "train":
    samples = pd.read_csv("vu-dmt-assigment-2-2023/training_set_VU_DM.csv")
else:
    samples = pd.read_csv("vu-dmt-assigment-2-2023/test_set_VU_DM.csv")


In [25]:
samples.info()
samples.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   date_time                    object 
 2   site_id                      int64  
 3   visitor_location_country_id  int64  
 4   visitor_hist_starrating      float64
 5   visitor_hist_adr_usd         float64
 6   prop_country_id              int64  
 7   prop_id                      int64  
 8   prop_starrating              int64  
 9   prop_review_score            float64
 10  prop_brand_bool              int64  
 11  prop_location_score1         float64
 12  prop_location_score2         float64
 13  prop_log_historical_price    float64
 14  position                     int64  
 15  price_usd                    float64
 16  promotion_flag               int64  
 17  srch_destination_id          int64  
 18  srch_length_of_stay          int64  
 19  

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [26]:
samples.isna().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4706481
visitor_hist_adr_usd           4705359
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4640941
orig_destination_distance

In [27]:
samples["date_time"] = pd.to_datetime(samples["date_time"])
samples['date_time'] = [datetime.timestamp(x) for x in samples["date_time"]]

Training samples and the test samples have almost the same amount of rows.

# Undersample our data

In [28]:
# Plot the counts
def plot_data(df, is_float=False):
    num_rows = len(df.columns) // 2 + len(df.columns) % 2

    fig, axs = plt.subplots(num_rows, 2, figsize=(10, num_rows*3))

    for i, col in enumerate(df.columns):
        ax = axs.flatten()[i]
        if not is_float:
            sns.histplot(df[col], ax=ax)
        else:
            sns.scatterplot(x=df.index, y=df[col], ax=ax)
        ax.set_title(col)

    # Remove unused subplots
    if len(df.columns) % 2 != 0:
        fig.delaxes(axs.flatten()[-1])

    plt.tight_layout()
    plt.show()

In [29]:
# Undersample_data
def undersample_data(df, column, alpha):
    X_train, y_train = df.drop(column, axis=1), df[column]
    smote = RandomUnderSampler(random_state=27, sampling_strategy=alpha)
    X, y = smote.fit_resample(X_train, y_train)
    print("Before undersampling: ", Counter(y_train))
    print("After undersampling: ", Counter(y))
    oversampled_df = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis=1)
    return oversampled_df

# Undersample multi data
def undersample_multi_data(df, column, alpha=4):
    X_train, y_train = df.drop(column, axis=1), df[column]
    a, b = Counter(df[column]).most_common()[0]

    # Create a dictionary with the desired sample count for class 4
    sampling_strategy = {}
    sampling_strategy[a] = round(b/alpha)

    rus = RandomUnderSampler(random_state=27, sampling_strategy=sampling_strategy)
    X, y = rus.fit_resample(X_train, y_train)
    print("Before undersampling: ", Counter(y_train))
    print("After undersampling: ", Counter(y))

    oversampled_df = pd.concat([pd.DataFrame(X, columns=X_train.columns), pd.DataFrame(y, columns=[column])], axis=1)
    return oversampled_df

# Missing Data

In [30]:
def fill_columns(df):
    new_df = df

    # Some variables we can set to -1
    set_zero = ["prop_review_score", "srch_query_affinity_score", "prop_location_score2"]
    new_df[set_zero] = new_df[set_zero].fillna(-1)

    if mode == "train":
        new_df["gross_bookings_usd"] = new_df["gross_bookings_usd"].fillna(0)

    # Fill missing competitor data with zeros
    cols = [c for c in new_df.columns if "comp" in c]
    new_df[cols] = new_df[cols].fillna(0)

    to_fill = ["visitor_hist_adr_usd", "visitor_hist_starrating"]

    # Calculate mean distances for each combination of visitor and property
    mean_distances = new_df.groupby(['visitor_location_country_id', 'prop_id'])['orig_destination_distance'].mean().fillna(-1).to_dict()

    # Replace NaN distances with corresponding means
    new_df['orig_destination_distance'] = new_df.apply(
        lambda row: mean_distances[(row['visitor_location_country_id'], row['prop_id'])]
                    if pd.isnull(row['orig_destination_distance'])
                    else row['orig_destination_distance'],
        axis=1
    )

    # Fill the remaining visitor_hist_adr_usd missing for a date_time, it will be filled with the last occurred value prior to that date_time
    new_df["visitor_hist_adr_usd"] = new_df["visitor_hist_adr_usd"].fillna(0)
    new_df["visitor_hist_starrating"] = new_df["visitor_hist_starrating"].fillna(-1)

    return new_df

In [31]:
def down_sample_negative_values(df, col_name, fr=0.5):
    # Assume df is your DataFrame and 'column_name' is the name of the column you want to undersample
    minus_one_rows = df[df[col_name] == -1]
    other_rows = df[df[col_name] != -1]

    # Undersample the -1 rows: let's say we want to keep half of them
    undersampled_minus_one_rows = minus_one_rows.sample(frac=fr, random_state=42)

    # Concatenate the undersampled -1 rows with the other rows
    undersampled_df = pd.concat([undersampled_minus_one_rows, other_rows])
    
    print("Before: ", len(minus_one_rows))
    print("After: ", len(undersampled_minus_one_rows))
    return undersampled_df

In [32]:
cleaner_samples = fill_columns(samples)

In [33]:
cleaner_samples = cleaner_samples.drop("visitor_hist_starrating",axis=1)

In [34]:
if mode == "train":
    # Downsample the booking_bool column
    undersampled = undersample_data(cleaner_samples, "booking_bool", 0.7)

    # Multi downsampling
#     to_under_sample = [("site_id", 2), ("prop_country_id", 2), ("srch_room_count", 2)]
#     end_df = undersampled
#     for under, alpha in to_under_sample:
#         end_df = undersample_multi_data(end_df, under, alpha)

# # Downsample negative values
# abc = [("visitor_hist_starrating", 0.01), ("visitor_hist_adr_usd", 0.1)]
# new_df = cleaner_samples.copy()
# for a,b in abc:
#     new_df = down_sample_negative_values(new_df, a, b)

Before undersampling:  Counter({0: 4819957, 1: 138390})
After undersampling:  Counter({0: 197700, 1: 138390})


In [35]:
if mode == "train" and not engineer_mode:
    dff=end_df
    transfer = ["prop_review_score"]
    float_cols = dff.select_dtypes(include=["float64"]).drop(transfer, axis=1).columns
    cat_cols = pd.concat([dff.select_dtypes(include=["int64"]), dff[transfer]]).columns
    plot_data(dff[cat_cols])
    plot_data(dff[float_cols], True)

In [36]:
if mode == "train" and not engineer_mode:
    def plot_correlation(df):
        plt.figure(figsize = (24, 12))
        corr = df.corr()
        sns.heatmap(corr, annot = True, linewidths = 1)
        plt.show()
        return corr['booking_bool'].abs().sort_values(ascending = False)

    res = plot_correlation(cleaner_samples)
    res

WORKS KINDA:
    
    - prop_location_score1, prop_location_score2: CORRELATION - prop_score
    - gross_bookings_usd, srch_length_of_stay: CORRELATION - booking_stay

NOT WORKING:

    - prop_starrating, prop_review_score: LIGHT CORRELATION
    - prop_country_id, visitor_location_country_id: CORRELATION
    - visitor_hist_adr_usd, visitor_hist_starrating: BIG CORRELATION, DOESNT WORK
    - srch_room_count, srch_adults_count: CORRELATION, intuitively not really a predictor or booking_bool
    - srch_saturday_night_bool, srch_length_of_stay: INVERSE CORRELATION, searching doesnt predict booking per se

In [37]:
def create_df_new_features(df):
    # Assumption: prop_log_historical_price is per night
    # Difference between user's historical prices and the current hotel
    df["user_hotel_price"] = df["visitor_hist_adr_usd"]-np.exp(df["prop_log_historical_price"])
    
    # Total property desirability score
    df["desirability"] = df["prop_location_score2"]+df["prop_location_score1"]+df["prop_review_score"]

    # Desirability to price per night ratio
    # High desirability and low total price is perfect
    df["d_p_ratio"] = df["desirability"] / df["price_usd"]
    
    # People per room
    df["people_per_room"] = (df["srch_adults_count"]+df["srch_children_count"])/df["srch_room_count"]

    # Convert to datetime data type
#     df["date_time"] = [datetime.fromtimestamp(x) for x in df["date_time"]]
#     df['date_time'] = pd.to_datetime(df['date_time'])
#     df['hour_id'] = df['date_time'].dt.hour # Extract day
#     df['day_id'] = df['date_time'].dt.day # Extract day    
#     df['season_id'] = df['date_time'].dt.quarter # Extract season
    
#     comp_col = [col for col in df.columns if col.endswith('rate')]
#     dat_comp = df[comp_col]
#     dat_comp_cheap = dat_comp[(dat_comp<0)].dropna(axis = 0, how = 'all').index
#     dat_comp_exp = dat_comp[(dat_comp>0)].dropna(axis = 0, how = 'all').index
#     dat_comp_same = dat_comp[(dat_comp==0)].dropna(axis = 0, how = 'all').index
    
#     dat_comp['comp_dat'] = 0
#     dat_comp.loc[dat_comp_cheap,'comp_dat'] = 1
#     dat_comp.loc[dat_comp_same,'comp_dat'] = 0
#     dat_comp.loc[dat_comp_exp,'comp_dat'] = 1
#     dat_comp.loc[dat_comp_cheap,'comp_dat'] = -1
#     df['comp_rate_all'] = dat_comp['comp_dat']
    
    # Overall competitor score
    # Total difference of the advantage according to the price
    
    df["overall_advantage"] = 0
    for i in range(1, 9):
        rat = "comp"+str(i)+"_rate"
        inv = "comp"+str(i)+"_inv"
        dff = "comp"+str(i)+"_rate_percent_diff"
        df["overall_advantage"] += df[rat]*(df[dff]/100)*df[inv]*df["price_usd"] 
    
    # Remove all the comp cols
    rates = [c for c in df.columns if c.endswith("rate")]
    invs = [c for c in df.columns if c.endswith("inv")]
    diffs = [c for c in df.columns if c.endswith("diff")]

    df = df.drop(["date_time", "srch_adults_count", "srch_children_count", "srch_room_count"], axis=1)
    df = df.drop(["prop_location_score2", "prop_location_score1", "prop_review_score"] , axis=1)
    df = df.drop(["visitor_hist_adr_usd", "prop_log_historical_price"], axis=1)
    df = df.drop(rates+invs+diffs, axis=1)
    return df

clean_training2 = create_df_new_features(undersampled if mode == "train" else cleaner_samples)
    

From our analysis we can see that price_usd and srch_length_of_stay have some outliers. 

## Down- and upsampling the training dataset
For some data points we need to downsample and for some we need to upsample the data. i.e. when we look at booking bool, that checks whether a hotel is booked, a lot of hotels have not been booked. This would introduce a big bias towards the non-booking side.

In [15]:
def find_outliers_and_normalize(end_df):
    # Filter out non-numeric columns
    df_numeric = end_df

    for column in df_numeric.select_dtypes(include=['float64']):
        Q1 = df_numeric[column].quantile(0.25)
        Q3 = df_numeric[column].quantile(0.75)
        IQR = Q3 - Q1

        # Define a mask for values outside the IQR
        mask = ((df_numeric[column] < (Q1 - 1.5 * IQR)) | (df_numeric[column] > (Q3 + 1.5 * IQR)))

        # Replace outliers with NaN
        df_numeric.loc[mask, column] = np.nan

    # Impute/Replace NaN values with means
    df_numeric.fillna(df_numeric.mean(), inplace=True)

    # Normalize all data
    id_cols = [c for c in end_df if "id" in c]
    non_id_df = df_numeric.drop(id_cols, axis=1) # remove the ids

    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(non_id_df), columns=non_id_df.columns)
    return pd.concat([df_scaled, end_df[id_cols]], axis=1) # add ids back

res = find_outliers_and_normalize(clean_training2)

# Adjust proportions of # of properties according to srch_id

In [16]:
# Get the rows that add up as close as possible to the target_sum
def select_rows(df, column, target_sum):
    df = df.sample(frac=1).reset_index(drop=True)  # Randomly shuffle the rows
    temp_sum = 0
    selected_rows = []
    for index, row in df.iterrows():
        if temp_sum + row[column] <= target_sum:
            temp_sum += row[column]
            selected_rows.append(index)
        if temp_sum >= target_sum:
            break
    return df.loc[selected_rows]

# cdf = pcad_df.groupby("prop_id")["srch_id"].nunique().reset_index()
# cdf.columns = ["prop_id", 'counts']
# smaller = cdf[cdf["counts"]<5]
# smaller_selected = select_rows(smaller, "counts", 1000)
# smaller_selected.sum()

In [17]:
def adjust_proportions(pcad_df, col1, col2, col1_ratio=0.9, col2_ratio=0.1, total_records=100000):
    count_df = pcad_df.groupby(col1)[col2].nunique().reset_index()
    count_df.columns = [col1, 'count']
    bigger = count_df[count_df["count"]>=5]
    smaller = count_df[count_df["count"]<5]

    # Calculate the desired total based on 'bigger' representing 90%
    desired_total_len = len(bigger) /  0.9

    # Calculate the desired number of 'smaller' samples to reach 10% of total
    desired_smaller_len = int(desired_total_len  * 0.10)

    # If 'smaller' represents more than 10% of the total, downsample it
    if len(smaller) > desired_smaller_len:
        smaller = smaller.sample(n=desired_smaller_len, random_state=42)

    # Check new proportions
    ls = len(smaller)
    lb = len(bigger)
    total_len = ls + lb

    print("smaller: ", ls, "proportion: ", ls/total_len)
    print("bigger: ", lb, "proportion: ", lb/total_len)
    
    # Get prop_ids in smaller and bigger
    smaller_prop_ids = smaller[col1].values
    bigger_prop_ids = bigger[col1].values

    # Get rows in pcad_df that match prop_ids in smaller and bigger
    smaller_rows = pcad_df[pcad_df[col1].isin(smaller_prop_ids)]
    bigger_rows = pcad_df[pcad_df[col1].isin(bigger_prop_ids)]

    # Combine back into a single dataframe
    new_pcad_df = pd.concat([smaller_rows, bigger_rows], ignore_index=True)
    return new_pcad_df

# specify the columns you want to proportionate, and the ratios
# Specify how many total_records you want
adjusted = adjust_proportions(res, "prop_id", "srch_id", 0.9, 0.1, total_records=100000)

smaller:  10214 proportion:  0.09999510499779725
bigger:  91931 proportion:  0.9000048950022027


In [18]:
# PCA if necessary
def pca(df):
    cols_to_scale = ['prop_starrating', "price_per_night", "d_p_ratio",
                     'srch_length_of_stay', "price_usd", "desirability", "user_hotel_price", 
                     "overall_advantage", "people_per_room"]

    df_to_pca, df_other = df[cols_to_scale], df.drop(cols_to_scale, axis=1)
    
    # Do PCA only on the cols_to_scale these are specified
    pca = PCA()
    dat = pca.fit_transform(df_to_pca)
    explained_variance = pca.explained_variance_ratio_
    var_dat = pd.DataFrame({'variance' : explained_variance})
    plt.bar(var_dat.index,var_dat['variance'])
    plt.show()

    pcas = var_dat.cumsum().sort_values("variance", ascending=False)

    cumulative_variance = var_dat.cumsum()
    print(cumulative_variance)

    # Find the number of components needed to capture 95% of the variance
    n_components = len(cumulative_variance[cumulative_variance <= 0.95].dropna())

    # Fit PCA again with the optimal number of components
    pca = PCA(n_components=n_components)
    dat = pca.fit_transform(df_to_pca)
    var_dat = pd.DataFrame({'variance' : explained_variance})
    var_dat.cumsum()

    pca_df = pd.concat([pd.DataFrame(dat), df_other], axis=1)
    return pca_df

In [19]:
# pcad_df = pca(adjusted)
pcad_df = adjusted

In [20]:
pcad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898753 entries, 0 to 4898752
Data columns (total 21 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   prop_starrating              float64
 1   prop_brand_bool              float64
 2   price_usd                    float64
 3   promotion_flag               float64
 4   srch_length_of_stay          float64
 5   srch_booking_window          float64
 6   srch_saturday_night_bool     float64
 7   srch_query_affinity_score    float64
 8   orig_destination_distance    float64
 9   random_bool                  float64
 10  user_hotel_price             float64
 11  desirability                 float64
 12  d_p_ratio                    float64
 13  people_per_room              float64
 14  overall_advantage            float64
 15  srch_id                      int64  
 16  site_id                      int64  
 17  visitor_location_country_id  int64  
 18  prop_country_id              int64  
 19  

In [21]:
# Cleaned data
if mode == "train":
    pcad_df.to_csv("100k_train_data.csv")
else:
    pcad_df.sample(n=100000).to_csv("100k_test_data.csv")

# Get 10k of samples from the final df
small_sample = pcad_df.sample(n=10000, random_state=1)
small_sample.to_csv("10k_train.csv" if mode == "train" else "10k_test_data.csv")